In [ ]:
from typing import Optional
from pennylane import numpy as np
import pennylane as qml

In [ ]:
from qml_essentials.ansaetze import Ansaetze

all_ansaetze = Ansaetze.get_available()

for ansatz in all_ansaetze:
    print(ansatz.__name__)

No_Ansatz
Circuit_1
Circuit_2
Circuit_3
Circuit_4
Circuit_6
Circuit_9
Circuit_10
Circuit_15
Circuit_16
Circuit_17
Circuit_18
Circuit_19
No_Entangling
Strongly_Entangling
Hardware_Efficient
GHZ


In [ ]:
from qml_essentials.topologies import Topology, Block

class MyHardwareEfficient(Circuit):
    @staticmethod
    def structure():
        return (
            Block(gate=Gates.RY),
            Block(gate=Gates.RZ),
            Block(gate=Gates.RY),
            Block(gate=Gates.CX, topology=Topology.brick_layer_wrap),
        )


from qml_essentials.model import Model

model = Model(
    n_qubits=2,
    n_layers=1,
    circuit_type=MyHardwareEfficient,
)

In [ ]:
from qml_essentials.gates import Gates, PulseInformation as pinfo
from qml_essentials.ansaetze import Circuit


class MyHardwareEfficient(Circuit):
    @staticmethod
    def n_params_per_layer(n_qubits: int) -> int:
        return n_qubits * 3

    @staticmethod
    def n_pulse_params_per_layer(n_qubits: int) -> int:
        n_params_RY = pinfo.num_params("RY")
        n_params_RZ = pinfo.num_params("RZ")
        n_params_CZ = pinfo.num_params("CZ")

        n_pulse_params = (num_params_RY + num_params_RZ) * n_qubits
        n_pulse_params += num_params_CZ * (n_qubits - 1)

        return pulse_params

    @staticmethod
    def get_control_indices(n_qubits: int) -> Optional[np.ndarray]:
        return None

    @staticmethod
    def build(w: np.ndarray, n_qubits: int, **kwargs):
        w_idx = 0
        for q in range(n_qubits):
            Gates.RY(w[w_idx], wires=q, **kwargs)
            w_idx += 1
            Gates.RZ(w[w_idx], wires=q, **kwargs)
            w_idx += 1

        if n_qubits > 1:
            for q in range(n_qubits - 1):
                Gates.CZ(wires=[q, q + 1], **kwargs)


from qml_essentials.model import Model

model = Model(
    n_qubits=2,
    n_layers=1,
    circuit_type=MyHardwareEfficient,
)

In [ ]:
noise_params = {
    "BitFlip": 0.0,
    "PhaseFlip": 0.0,
    "AmplitudeDamping": 0.0,
    "PhaseDamping": 0.0,
    "Depolarizing": 0.0,
    "MultiQubitDepolarizing": 0.0,
}


class MyNoisyHardwareEfficient(Circuit):
    @staticmethod
    def n_params_per_layer(n_qubits: int) -> int:
        return n_qubits * 3

    @staticmethod
    def get_control_indices(n_qubits: int) -> Optional[np.ndarray]:
        return None

    @staticmethod
    def build(w: np.ndarray, n_qubits: int, noise_params=None):
        w_idx = 0
        for q in range(n_qubits):
            Gates.RY(w[w_idx], wires=q, noise_params=noise_params)
            w_idx += 1
            Gates.RZ(w[w_idx], wires=q, noise_params=noise_params)
            w_idx += 1

        if n_qubits > 1:
            for q in range(n_qubits - 1):
                Gates.CZ(wires=[q, q + 1], noise_params=noise_params)


model(
    model.params,
    inputs=None,
    execution_type="density",
    noise_params={
        "BitFlip": 0.01,
        "PhaseFlip": 0.02,
        "AmplitudeDamping": 0.03,
        "PhaseDamping": 0.04,
        "Depolarizing": 0.05,
        "MultiQubitDepolarizing": 0.06,
    },
)

/home/maja/code/qml-essentials/qml_essentials/model.py:429: UserWarning: density measurement does ignore output_qubit, which is [0, 1].
  warnings.warn(


Array([[ 0.16524883+1.85197812e-17j,  0.02873437+2.19032164e-01j,
         0.07240738+2.23751414e-01j, -0.10604392+5.48367926e-02j],
       [ 0.02873437-2.19032164e-01j,  0.33952285+2.12697759e-18j,
         0.33902759-6.26207211e-02j,  0.06369996+1.75706070e-01j],
       [ 0.07240738-2.23751414e-01j,  0.33902759+6.26207211e-02j,
         0.38699394+1.25534402e-17j,  0.0328239 +1.96940052e-01j],
       [-0.10604392-5.48367926e-02j,  0.06369996-1.75706070e-01j,
         0.0328239 -1.96940052e-01j,  0.10823439+1.57663123e-17j]],      dtype=complex128)